Задание 5. Web-scraping на языке Python

Инструкция:

Шаг 1.  Изучите материалы лекционных и практических занятий по темам 4.1 и 4.2.

Шаг 2. Используя доступ к данным по API, соберите с [ресурса](https://trudvsem.ru/) данные по интересующим Вас вакансиям или вакансиям Вашего региона.
Дополнительную информацию  по данному API можно найти [здесь](https://trudvsem.ru/opendata/api). Количество записей и набор признаков определяете самостоятельно. Собранные данные сохраните в DataFrame.  

Шаг 3. С использованием библиотеки BeautifulSoup выполните web-scraping [ресурса](https://prodoctorov.ru/). Населенный пункт и специализацию врача выбрать самостоятельно.  Количество записей и набор признаков определяете самостоятельно. Собранные данные сохраните в DataFrame.

Шаг 4.  Опубликуйте файл расширения ipynb на платформе Odin.

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [ ]:
# Сбор вакансий 78 региона
df = pd.DataFrame(columns = ['название_вакансии','заработная_плата','регион','код_региона','название_компании'])

for page in range (0,179):
    par = {'offset' : page,
          'limit': 100}
    answer = requests.get("http://opendata.trudvsem.ru/api/v1/vacancies/region/78", params= par)
    answer.status_code
    answer.text
    dct_vacansy = answer.json()
    dct_vacansy.keys()
    total_record = dct_vacansy['meta']['total']

    # анализ структуры полученной вакансии

    dct_vacansy['results']['vacancies'][0]['vacancy']
    dct_vacansy['results']['vacancies'][1]['vacancy']['region']['name']
    dct_vacansy['results']['vacancies'][1]['vacancy']['region']['region_code']
    dct_vacansy['results']['vacancies'][1]['vacancy']['company']['name']
    dct_vacansy['results']['vacancies'][1]['vacancy']['salary_min']
    dct_vacansy['results']['vacancies'][1]['vacancy']['creation-date']
    dct_vacansy['results']['vacancies'][0]['vacancy']
    dct_vac = {'название_вакансии': None,
              'заработная_плата': None,
              'регион': None,
              'код_региона': None,
              'название_компании': None,
                }
    lst_vac = []
    for vacansy in dct_vacansy['results']['vacancies']:
        dct_vac['название_вакансии'] = vacansy['vacancy']['job-name']
        dct_vac['заработная_плата'] = vacansy['vacancy']['salary_min']
        dct_vac['регион'] = vacansy['vacancy']['region']['name']
        dct_vac['код_региона'] = vacansy['vacancy']['region']['region_code'].replace ('0', '')
        dct_vac['название_компании'] = vacansy['vacancy']['company']['name']
        lst_vac.append(dct_vac.copy())
    df = pd.concat([df, pd.DataFrame(lst_vac)], ignore_index=True)
df

,название_вакансии,заработная_плата,регион,код_региона,название_компании
0,Data analyst / Аналитик данных,0,Город Санкт-Петербург,78,"ООО ""КОЛЛ СОЛЮШЕНС"""
1,Уборщица,48300,Город Санкт-Петербург,78,"ООО ""ЛАЙФ ЭНЕРДЖИ САНКТ-ПЕТЕРБУРГ"""
2,Торговый агент,38550,Город Санкт-Петербург,78,"АО ""ВБД"""
3,Торговый агент,38550,Город Санкт-Петербург,78,"АО ""ВБД"""
4,Ночной повар,55000,Город Санкт-Петербург,78,"ООО ""МОЙКА 22"""
...,...,...,...,...,...
17826,Слесарь-сборщик,40000,Город Санкт-Петербург,78,"ООО ""МЕТАЛЛДИЗАЙН"""
17827,администратор приёмки,60000,Город Санкт-Петербург,78,"ООО ""Корзина"""
17828,Бухгалтер,60000,Город Санкт-Петербург,78,"ФГБУ ""МСГ"""
17829,Учитель английского языка,34000,Город Санкт-Петербург,78,ГБОУ СОШ №551 КИРОВСКОГО РАЙОНА САНКТ-ПЕТЕРБУРГА


In [ ]:
print (dct_vacansy['meta']) # здесь содержится общее количество записей

{'total': 17831, 'limit': 100}


In [ ]:
# Сбор данных о шуруповертах

In [ ]:
df = pd.DataFrame(columns = ['название шуруповерта','цена','наличие'])
link = "https://tandemtools.ru/catalog/shurupovertyi/?page="
for page in list(range(0,57)):
    print (page)
    # выводим счетчик страниц, чтобы в случае поломки знать где искать
    url = link + str(page)
    anwers = requests.get(url)
    soup = BeautifulSoup(anwers.text, 'lxml')
    drels = soup.find_all('div',
                          attrs={'class': ['product-card js-item-card with-discount',
                                           'product-card js-item-card','product-card js-item-card not-orderable']})

    dct_drels = {'название шуруповерта': None,
                 'цена': None,
                 'наличие': None
                 }

    lst_drels = []

    for drel in drels:
        dct_drels['название шуруповерта'] = drel.find('a', attrs={'class': 'ttl'}).get_text(strip=True)
        price = drel.find('span', attrs={'class': ['price',
                                               'price hot-price', 'val']})
        if price != None:
          dct_drels['цена'] = price.get_text(strip=True)
        else:
          dct_drels['цена'] = 'нет цены'
        order = drel.find('span', attrs={'class': ['val']})
        if (order != None) | (price == None) :
          dct_drels['наличие'] = 'нет в наличии'
        else:
          dct_drels['наличие'] = 'в наличии'
        lst_drels.append(dct_drels.copy())

    df = pd.concat([df, pd.DataFrame(lst_drels)], ignore_index=True)
    time.sleep(10)
df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56


,название шуруповерта,цена,наличие
0,Шуруповерт EDON DS-350 сетевой,1 579,в наличии
1,"Шуруповерт сетевой ЭШ-0,6 Диолд",2 369,в наличии
2,Шуруповерт сетевой Интерскол ДШ-10/260Э2,3 229,в наличии
3,Шуруповерт SKIL сетевой 6940MK 520Вт,10 449,в наличии
4,Аккумуляторная дрель-шуруповерт DeWalt DCD 710 C2,9 739,в наличии
...,...,...,...
669,Дрель-шуруповерт ВИХРЬ ДА-18-2К,8 710,нет в наличии
670,"Дрель-шуруповерт ВИХРЬ ДА-14,4-2К",23 390,нет в наличии
671,"Дрель-шуруповерт ВИХРЬ ДА-14,4-2",2 301,нет в наличии
672,Дрель-шуруповерт ВИХРЬ ДА-12-2,1 631,нет в наличии


""
